In [1]:
import pandas as pd
import jieba.posseg as pseg
import keras

train = pd.read_csv('/home/au/文件/Data/NLP Data/train.csv', encoding = 'utf-8')
train = train.astype(str)
cols = ['title1_zh', 'title2_zh', 'label']
train = train.loc[0:100, cols]
# train.head(3)

def jieba_tokenizer(text):
    words = pseg.cut(text)
    return ' '.join([
        word for word, flag in words if flag != 'x'])

train['title1_tokenized'] = \
    train.loc[0:, 'title1_zh'] \
         .apply(jieba_tokenizer)
train['title2_tokenized'] = \
    train.loc[0:, 'title2_zh'] \
         .apply(jieba_tokenizer)

train.iloc[0:, [0, 3]].head()
train.iloc[0:, [1, 4]].head()

/home/au/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.873 seconds.
Prefix dict has been built succesfully.


,title2_zh,title2_tokenized
0,警方辟谣“鸟巢大会每人领5万” 仍有老人坚持进京,警方 辟谣 鸟巢 大会 每人 领 5 万 仍 有 老人 坚持 进京
1,深圳GDP首超香港？深圳统计局辟谣：只是差距在缩小,深圳 GDP 首 超 香港 深圳 统计局 辟谣 只是 差距 在 缩小
2,GDP首超香港？深圳澄清：还差一点点……,GDP 首 超 香港 深圳 澄清 还 差 一点点
3,去年深圳GDP首超香港？深圳统计局辟谣：还差611亿,去年 深圳 GDP 首 超 香港 深圳 统计局 辟谣 还 差 611 亿
4,吃了30年食用油才知道，一片大蒜轻松鉴别地沟油,吃 了 30 年 食用油 才 知道 一片 大蒜 轻松 鉴别 地沟油


maxword = 10000
tokenizer = keras \
    .preprocessing \
    .text \
    .Tokenizer(num_words=maxword)

In [2]:
corpus_x1 = train.title1_tokenized
corpus_x2 = train.title2_tokenized
corpus = pd.concat([
    corpus_x1, corpus_x2])
corpus.shape
print(corpus)

0               2017 养老保险 又 新增 两项 农村 老人 人人 可 申领 你 领到 了 吗
1        你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港
2        你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港
3        你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港
4                             用 大蒜 鉴别 地沟油 的 方法 怎么 鉴别 地沟油
5        你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港
6                                      吃 榴莲 的 禁忌 吃 错会 致命
7        你 不 来 深圳 早晚 你 儿子 也 要 来 不出 10 年 深圳 人均 GDP 将 超 香港
8              旅行 青蛙 居然 是 一款 生育 意愿 测试 器 大家 还是 玩 珠宝 V 课 吧
9                             用 大蒜 鉴别 地沟油 的 方法 怎么 鉴别 地沟油
10                            用 大蒜 鉴别 地沟油 的 方法 怎么 鉴别 地沟油
11                            用 大蒜 鉴别 地沟油 的 方法 怎么 鉴别 地沟油
12                            用 大蒜 鉴别 地沟油 的 方法 怎么 鉴别 地沟油
13                            用 大蒜 鉴别 地沟油 的 方法 怎么 鉴别 地沟油
14             飞机 就要 起飞 一个 男人 在 机舱 口 跪下 这 是 今天 最 催泪 的 一幕
15                飞机 就要 起飞 一个 男人 在 机舱 口 跪下 这 是 最 催泪 的 一幕
16                           男人 在 机舱 口 跪下 原来 一切 都 只 因为 爱
17             飞机 就要 起飞 一个 男人 在

In [3]:
MAX_NUM_WORDS = 10000
tokenizer = keras \
    .preprocessing \
    .text \
    .Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(corpus)
x1_train = tokenizer \
    .texts_to_sequences(corpus_x1)
x2_train = tokenizer \
    .texts_to_sequences(corpus_x2)
len(x1_train)
x1_train[:1]

[[318, 319, 234, 320, 321, 322, 235, 323, 184, 324, 9, 325, 12, 27]]

In [4]:
for seq in x1_train[:1]:
    print([tokenizer.index_word[idx] for idx in seq])

['2017', '养老保险', '又', '新增', '两项', '农村', '老人', '人人', '可', '申领', '你', '领到', '了', '吗']


In [5]:
for seq in x1_train[:10]:
    print(len(seq), seq[:5], ' ...')

14 [318, 319, 234, 320, 321]  ...
19 [9, 78, 59, 29, 120]  ...
19 [9, 78, 59, 29, 120]  ...
19 [9, 78, 59, 29, 120]  ...
9 [55, 52, 28, 25, 1]  ...
19 [9, 78, 59, 29, 120]  ...
7 [2, 236, 1, 124, 2]  ...
19 [9, 78, 59, 29, 120]  ...
16 [97, 328, 237, 64, 329]  ...
9 [55, 52, 28, 25, 1]  ...


In [6]:
max_seq_len = max([
    len(seq) for seq in x1_train])
max_seq_len

22

In [7]:
MAX_SEQUENCE_LENGTH = 20
x1_train = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_train, 
                   maxlen=MAX_SEQUENCE_LENGTH)

x2_train = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_train, 
                   maxlen=MAX_SEQUENCE_LENGTH)

In [12]:
x1_train[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 53, 47, 39, 56, 62, 79, 46,
       80, 81, 12], dtype=int32)

In [8]:
for seq in x1_train + x2_train:
    assert len(seq) == 20
    
print("所有新聞標題的序列長度皆為 20 !")

所有新聞標題的序列長度皆為 20 !


In [13]:
x1_train[:5]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  53,  47,  39,
         56,  62,  79,  46,  80,  81,  12],
       [  0,   0,   0,   0,   0,   0,   0,  87,  66,  88,  68,  30,  41,
         48,  49,   5,   7,  40,  63,  89],
       [  0,   0,   0,   0,   0,   0, 202, 203, 204, 205, 206,  51,  10,
        207, 208, 108, 130, 209, 210, 108],
       [  0,   0,   0,   0, 102, 186,  70,  26, 187,  71, 103, 188, 103,
          1, 104,  65, 143,   9, 144,  27],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 154, 111,  85,
         86, 155,   2,  69,  40, 156, 124]], dtype=int32)

In [9]:
import numpy as np 

# 定義每一個分類對應到的索引數字
label_to_index = {
    'unrelated': 0, 
    'agreed': 1, 
    'disagreed': 2
}

# 將分類標籤對應到剛定義的數字
y_train = train.label.apply(
    lambda x: label_to_index[x])

y_train = np.asarray(y_train) \
            .astype('float32')

y_train[:5]

array([0., 0., 0., 0., 1.], dtype=float32)

In [10]:
y_train = keras \
    .utils \
    .to_categorical(y_train)

y_train[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [11]:
from sklearn.model_selection \
    import train_test_split

VALIDATION_RATIO = 0.1
# 小彩蛋
RANDOM_STATE = 9527

x1_train, x1_val, \
x2_train, x2_val, \
y_train, y_val = \
    train_test_split(
        x1_train, x2_train, y_train, 
        test_size=VALIDATION_RATIO, 
        random_state=RANDOM_STATE
)

In [14]:
print("Training Set")
print("-" * 10)
print(f"x1_train: {x1_train.shape}")
print(f"x2_train: {x2_train.shape}")
print(f"y_train : {y_train.shape}")

print("-" * 10)
print(f"x1_val:   {x1_val.shape}")
print(f"x2_val:   {x2_val.shape}")
print(f"y_val :   {y_val.shape}")
print("-" * 10)
print("Test Set")

Training Set
----------
x1_train: (90, 20)
x2_train: (90, 20)
y_train : (90, 3)
----------
x1_val:   (11, 20)
x2_val:   (11, 20)
y_val :   (11, 3)
----------
Test Set


In [15]:
from keras import layers
embedding_layer = layers.Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM)

NameError: name 'NUM_EMBEDDING_DIM' is not defined